In [6]:
import numpy as np
from sparse_vector import *
from vector_polynomial_isomorphism import *
from minor_flattening import *

We note that we are using the base field `QQ` for all computations currently

In [7]:
# Let us initialize constants and vector spaces.

d = 7
N = d+1
n = [N]
indeterminates = form_indeterminates(n)
R = PolynomialRing(QQ, indeterminates)
R.inject_variables()

glN = MatrixSpace(QQ, N)
dimV = binomial(N + 2 - 1, 2)
V = VectorSpace(QQ, dimV)

vp_iso = vector_polynomial_isomorphism(R, V, indeterminates, 2)

Defining x_0, x_1, x_2, x_3, x_4, x_5, x_6, x_7


In [8]:
def x_(i):
    return var("".join(["x_", str(i)]))

delta = [[R(x_(i)*x_(j+1) - x_(j)*x_(i+1)) for j in range(d)] for i in range(d-1)]

Q = [[R(delta[i+2][j] - 2*delta[i+1][j+1] + delta[i][j+2]) for j in range(i+1, d-2)] for i in range(d-3)]

Q_lst = list(np.concatenate(Q).flat)
f_vec = [vp_iso.polynomial_to_vector(qi) for qi in Q_lst]

In [9]:
# Check tangential variety vanishes on Q_lst

R_parameterized = PolynomialRing(QQ, [var('t'), var('s')])
R_parameterized.inject_variables()
T_param = [1] + [R_parameterized(t^(i+1) + s*diff(t^(i+1), t)) for i in range(d)]
print(T_param)
for qi in Q_lst:
    print(qi(T_param))

Defining t, s
[1, t + s, t^2 + 2*t*s, t^3 + 3*t^2*s, t^4 + 4*t^3*s, t^5 + 5*t^4*s, t^6 + 6*t^5*s, t^7 + 7*t^6*s]
0
0
0
0
0
0
0
0
0
0


In [10]:
W = V.subspace(f_vec)
print(W)

Vector space of degree 36 and dimension 10 over Rational Field
Basis matrix:
[ 0  0  0  0  1  0  0  0  0  0 -4  0  0  0  0  3  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
[ 0  0  0  0  0  1  0  0  0  0  0 -3  0  0  0  0  2  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
[ 0  0  0  0  0  0  1  0  0  0  0  0  0  0  0  0  0 -9  0  0  0  8  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
[ 0  0  0  0  0  0  0  1  0  0  0  0  0  0  0  0  0  0 -6  0  0  0  5  0  0  0  0  0  0  0  0  0  0  0  0  0]
[ 0  0  0  0  0  0  0  0  0  0  0  0  1  0  0  0  0 -4  0  0  0  3  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
[ 0  0  0  0  0  0  0  0  0  0  0  0  0  1  0  0  0  0 -3  0  0  0  2  0  0  0  0  0  0  0  0  0  0  0  0  0]
[ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  1  0  0  0  0  0  0  0  0 -9  0  0  8  0  0  0  0  0  0  0  0  0]
[ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  1  0  0  0 -4  0  0  3  0  0  0  0  0  0  0  0  0]
[ 0  0  0  0  0  0  0  0  0  0  0  0  0  0 

In [11]:
W_perp = W.complement()
print(W_perp)
p = W.dimension()
q = W_perp.dimension()

Vector space of degree 36 and dimension 26 over Rational Field
Basis matrix:
26 x 36 dense matrix over Rational Field


In [12]:
def E_action(a, b, grad_fl):
    sum = -1*indeterminates[b]*grad_fl[a]
    return R(sum)

We note `M_f` $:= \tilde{M}_f$

In [13]:
g_sparse = [sparsify(W_perp.gen(k)) for k in range(q)]

In [14]:
E_action_fl = [[0 for j in range(N)] for i in range(N)]

# Construct M_f
M_f = zero_matrix(QQ, p*q, N^2, sparse=True)
for l in range(p):
    grad_fl = vp_iso.vector_to_polynomial(W.gen(l)).gradient()
    for i in range(N):
        for j in range(N):
            E_action_fl[i][j] = sparsify(vp_iso.polynomial_to_vector(E_action(i,j,grad_fl)))
    for k in range(q):
        for i in range(N):
            for j in range(N):
                M_f[(q*l) + k,(N*i) + j] = sparse_inner_product(E_action_fl[i][j], g_sparse[k])
                

In [15]:
ns = M_f.right_kernel()
print('rank', M_f.rank())
print('dim(ker(M_f3)) =', ns.dimension())


rank 60
dim(ker(M_f3)) = 4


In [16]:
print('expected: 4')

expected: 4
